In [19]:
import json
import pandas as pd
from konlpy.tag import  Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import numpy as np
import os

In [20]:
# 월별 데이터로 생성하기
json_list = os.listdir('../news-crawler/it/')
json_files = [file for file in json_list if file.endswith('.json')]  
data = []
df = pd.DataFrame()
for i in json_files:
    for line in open(('../news-crawler/it/'+i),"r", encoding='utf-8-sig'):
        df = pd.concat([df, pd.DataFrame(json.loads(line), columns=['id', 'title', 'content', 'date', 'like'])])  

In [21]:
# 주차별 데이터로 생성하기
# with open("../news-crawler/politics/2021-11-16-20.json", "r", encoding='utf-8-sig') as f:
#     tmp = json.load(f)
# df = pd.DataFrame(tmp, columns=['id', 'title', 'content', 'date', 'like'])

In [22]:
okt = Okt() 
# okt.analyze  #구(Phrase) 분석
# okt.morphs   #형태소 분석
# okt.nouns    #명사 분석
# okt.pos      #형태소 분석 태깅

noun_list = []
for content in tqdm(df['content']): 
    nouns = okt.nouns(content)
    noun_list.append(nouns)
df['nouns'] = noun_list
print(df.head())

100%|██████████| 6403/6403 [04:44<00:00, 22.50it/s]


                                                  id  \
0  {'company': '002', 'id': '0002219192', 'sid': ...   
1  {'company': '018', 'id': '0005087561', 'sid': ...   
2  {'company': '417', 'id': '0000755065', 'sid': ...   
3  {'company': '123', 'id': '0002259776', 'sid': ...   
4  {'company': '008', 'id': '0004671328', 'sid': ...   

                                            title  \
0                      이재명 "탄소 50% 감축", 그런데 탈원전은?   
1                   “이제 지쳤다” 황교익, 돌연 이재명에 선 긋는 이유   
2  국힘에게 '김종인'이란?… "결정적 킹메이커" vs "후보와 좋게 이별한 적 없어"   
3        野 "'대장동비리'를 2011년 尹이 수사로 막았어야? '특검' 받아라"   
4             '올블랙' 착장 인물, 김혜경 아니었다… 오보 삭제 후 정정보도   

                                             content                 date  \
0  "원전은 시한폭탄" → "원자력은 경제구조"\n더불어민주당 이재명 대선후보가 "탄소...  2021-11-16 19:44:01   
1  [이데일리 송혜수 기자] 경기관광공사 사장 내정 논란으로 후보자 자리에서 물러났던 ...  2021-11-16 19:40:01   
2  원본보기\n국민의힘 선거대책위원회에 김종인 전 비상대책위원장 영입이 유력하다고 여겨...  2021-11-16 19:38:00   
3  野 "조우형 '봐주기 수사'? 거짓공세 중단하고 특검부터 받

In [23]:
# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.3,min_samples=6, metric = "cosine")
# 거리 계산 식으로는 Cosine distance를 이용
result = model.fit_predict(vector)
df['result'] = result

In [24]:
# Delete noise or garbage data
df = df[df['result'] != -1]
df = df[df['result'] != 0]

cluster_dict = {}

# Create clustering using titles
for i in df['result'].unique().tolist():
    cluster_dict[i] = df[df['result'] == i].title.tolist()

# Order by Hot topic    
sorted_cluster_dict = sorted(cluster_dict.items(), key=lambda x : len(x[1]), reverse=True)

In [25]:
final_dict = {}

# Create 10 data sets of clustered data
for idx, (cluster_num, titles) in enumerate(sorted_cluster_dict[:10]):
    keywords_list = []
    id_list = []
    like_list = []

    # Extract keyword from title
    for title in titles:
        res = " ".join(titles).split()
        keywords = okt.phrases(title)
        keywords_list.extend(res)
        # Add like / ID for sorting and linking
        id_list = df[df['result']==cluster_num].id.tolist()
        like_list = df[df['result']==cluster_num].like.tolist()
    # Create keyword standard
    standard_keyword = set(keywords_list)
    print(standard_keyword)
    tmp_keyword = []

    # Keyword Counting
    for keyword in standard_keyword:
        tmp_keyword.append((keyword, keywords_list.count(keyword)))

    # Hot Keyword Top 3
    tmp_keyword.sort(key = lambda x : x[1], reverse=True)
    final_keyword = [keyword for keyword in tmp_keyword[:5]]
    print(final_keyword)
    # Create dictonary file
    final_data = []
    for id, title, like in zip(id_list, titles, like_list):
        id['like'] = like
        id['title'] = title
        final_data.append(id)
    final_dict[f"cluster{idx}"] = {}
    final_dict[f'cluster{idx}']['keyword'] = final_keyword
    final_dict[f'cluster{idx}']['data'] = final_data

{'李,', '언제까지?', "'사과'…호남", '분노', '가짜뉴스', '관훈클럽', "'최장", '"적개심의', "시간'", '이재명,', '김종인에', '비판', '연일', '이야기', '"이재명', '된', '"기민함', '직격', '통합', '상승세', "'백브리핑'", '지령"', '직격…"화천대유', '당정', '5·18', '오갔나', '조치"', "구애'…선대위", '언제쯤?', 'CCTV', '부인', '낙상사고', '남', '도울', '왜곡"', '때리기…"전형적', '채', '강력', '벌렸다', '부족"', '돼야"…야당', '특검', '윤석열', '키워"', '탓"', '후폭풍…"사실', '지속…이재명과', '더', '尹', '위험"', '허위사실', '"국민', "'밀당'", '유포', '풀릴까?', '대응"', '재개…"부인', '법적', '"수사', '뿌리는', '"우리가', '선대위', '수용', '수도"', '신경전도', '미진하면…尹도', '관련', '이재명', '"스스로', '정치"', "의혹'", '일정', '"계기되면', "'공개", '김혜경', "'괴소문'에", '해야"', '윤석열"', '논란', '김종인의', "'추격자'", "'산책", '조건부', '"좌표', '공개…이-윤', '與', '토론회…어떤', '여지…尹', '초보운전이', '지지율', '격차', '언론', '지지자에', '"음주운전보다', '분향', '못한', '대장동', '잇겠다"…송영길', '윤석열,'}
[('이재명', 60), ('윤석열', 45), ('이재명,', 30), ('더', 30), ('관련', 30)]
{'건조…친환경', '한국이', '할', '여정에', '우리가', '"탄소중립', "'한-모잠비크", '"최고', '강국"', '것"', "FLNG'", '고도화', '"전세계', '"FLNG,', '“세계는', '조선', '참석…"세계', '힘"', '여정', '대통령', '출항…文', '척', 'FLNG,', '탄소중립', '찾은'

In [26]:
final_dict

{'cluster0': {'keyword': [('이재명', 60),
   ('윤석열', 45),
   ('이재명,', 30),
   ('더', 30),
   ('관련', 30)],
  'data': [{'company': '055',
    'id': '0000933782',
    'sid': '100',
    'like': 17,
    'title': '\'산책 의혹\' 후폭풍…"사실 왜곡" "스스로 논란 키워"'},
   {'company': '055',
    'id': '0000933777',
    'sid': '100',
    'like': 1,
    'title': '"계기되면 도울 수도" 김종인의 여지…尹 선대위 언제쯤?'},
   {'company': '055',
    'id': '0000933775',
    'sid': '100',
    'like': 288,
    'title': '"기민함 부족" 이재명 연일 당정 때리기…"전형적 남 탓"'},
   {'company': '055',
    'id': '0000933581',
    'sid': '100',
    'like': 29,
    'title': '윤석열 지지율 상승세 지속…이재명과 격차 더 벌렸다'},
   {'company': '055',
    'id': '0000933568',
    'sid': '100',
    'like': 1367,
    'title': '이재명, 지지자에 "우리가 언론 돼야"…야당 "좌표 지령"'},
   {'company': '055',
    'id': '0000933567',
    'sid': '100',
    'like': 2,
    'title': "윤석열, 김종인에 '공개 구애'…선대위 '밀당' 언제까지?"},
   {'company': '055',
    'id': '0000933566',
    'sid': '100',
    'like': 67,
    'title': '\'추격자\' 된 이재명, 尹 직격

In [27]:
file_path = '../json_data/it/new_data.json'
with open(file_path, 'w', encoding='utf-8') as f:
	json.dump(final_dict, f, ensure_ascii=False)